In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
import glob
import os
from pathlib import Path

import matplotlib.pyplot as plt
import torch

from experiments.model_utils import load_medsam
from experiments.utils import (
    load_process_csv, process_slice, process_bbox_str, segment, split_seg, get_seg_bbox, \
    get_slices_filenames, segment_slice_sequence
)
from experiments.viz_utils import plot_results

device = 'cuda' if torch.cuda.is_available() else 'cpu'
medsam = load_medsam("../work_dir/MedSAM/medsam_vit_b.pth", device)

/home/liushifeng/Documents/GitHub/MedSAM/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [8]:
data_folder = Path("/home/liushifeng/Desktop/DeepLesion Dataset/")
df = load_process_csv(data_folder / "DL_info.csv")

batch_folders = [data_folder / f for f in os.listdir(data_folder) if f.startswith("Images_png_") and ".zip" not in f]

## Segment 2D slices

In [30]:
# %matplotlib inline
scan_folders = [Path(f) for f in glob.glob(str(data_folder / "Images_png_*/Images_png/*"))]

In [13]:
# %matplotlib inline
# every scan_folder contains N png files
for scan_folder in scan_folders:
    scan_name = scan_folder.stem
    print("Scan name:", scan_name)
    df_scan = df[df['scan_name'].eq(scan_name)]

    for i in range(len(df_scan)):
        df_slice = df_scan.iloc[i]
        k = df_slice['Key_slice_index']

        key_slice_path = scan_folder / df_slice['file_name']
        lung, abdomen = process_slice(key_slice_path, rgb=True)

        bbox = [round(x) for x in process_bbox_str(df_slice['Bounding_boxes'])]
        seg = segment(abdomen, bbox, medsam)
        segs = split_seg(seg)

        plot_results(abdomen, [bbox], segs, plot=True,
                     save_path=f"outputs/2d segs/{scan_name}_{k}")
        break
    break

NameError: name 'scan_folders' is not defined

## Extend to 3D

In [ ]:
for scan_folder in scan_folders:
    scan_name = scan_folder.stem
    print("Scan name:", scan_name)
    df_scan = df[df['scan_name'].eq(scan_name)]

    for i in range(len(df_scan)):
        df_slice = df_scan.iloc[i]
        print(df_slice)
        k = df_slice['Key_slice_index']
        slice_filenames = get_slices_filenames(df_slice['Slice_range'].split(", "))
        up_slice_paths = [scan_folder / f for f in slice_filenames if int(f.split(".")[0]) >= k]
        down_slice_paths = [scan_folder / f for f in slice_filenames if int(f.split(".")[0]) <= k][::-1]

        plot = False
        save = True
        up_slices, up_slice_segs = segment_slice_sequence(medsam, up_slice_paths, df_slice['bbox'], plot=plot, save=save)
        down_slices, down_slice_segs = segment_slice_sequence(medsam, down_slice_paths, df_slice['bbox'], plot=plot, save=save)
        break

Scan name: 004428_01_01
cont
Scan name: 004415_01_01
File_name                                                  004415_01_01_065.png
Patient_index                                                              4415
Study_index                                                                   1
Series_ID                                                                     1
Key_slice_index                                                              65
Measurement_coordinates       274.659, 205.36, 288.927, 212.208, 284.035, 20...
Bounding_boxes                               269.659, 199.137, 293.927, 219.328
Lesion_diameters_Pixel_                                         15.826, 11.3043
Normalized_lesion_location                         0.529507, 0.374532, 0.394307
Coarse_lesion_type                                                           -1
Possibly_noisy                                                                0
Slice_range                                                        

In [9]:
%matplotlib inline
plot = False
up_slices, up_slice_segs = segment_slice_sequence(medsam, up_slice_paths, df_slice['bbox'], plot=plot, save=True)
down_slices, down_slice_segs = segment_slice_sequence(medsam, down_slice_paths, df_slice['bbox'], plot=plot, save=True)

NameError: name 'up_slice_paths' is not defined

In [ ]:
import numpy as np
import pandas as pd

In [2]:
def get_slice_stats(slices, slice_segs):
    median_conf = []
    size = []
    intensity = []

    for i in range(len(slices)):
        img = np.array(slices[i])
        seg = slice_segs[i]
        low, mid, high = split_seg(seg)

        median_conf.append(np.median(seg[seg > 0.1]))
        size.append((mid.sum() + high.sum()))
        intensity.append(np.mean(img[high.astype(bool)]))

    # get stats of slices
    res = pd.DataFrame([median_conf, size, intensity]).T
    res.columns = ['median_conf', 'size', 'intensity']
    res = res / res.iloc[0]
    return res

In [4]:
res

NameError: name 'res' is not defined

In [3]:
res_up = get_slice_stats(up_slices, up_slice_segs)
res_down = get_slice_stats(down_slices, down_slice_segs)

res = pd.concat([res_up.iloc[1:].iloc[::-1], res_down], axis=0).reset_index(drop=True)
res.plot(figsize=(5,4));

NameError: name 'up_slices' is not defined

In [ ]:
import numpy as np

# Calculate the centroid of the segs[2] 2D mask
mask = segs[2]
y, x = np.where(mask == 1)
centroid = (np.mean(x), np.mean(y)) if len(x) > 0 and len(y) > 0 else (None, None)
print("Centroid:", centroid)

In [ ]:
plt.figure(figsize=(6, 5));
plt.imshow(abdomen);
plt.imshow(segs[2], cmap=transparent_cmap("red"));
plt.colorbar();
plt.axis('off');